In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
matches = pd.read_csv(r"D:\ML\IPL_win_probability\data\matches.csv")
deliveries = pd.read_csv(r"D:\ML\IPL_win_probability\data\deliveries.csv")

In [51]:
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [52]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [53]:
matches.columns

Index(['id', 'season', 'city', 'date', 'match_type', 'player_of_match',
       'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'result', 'result_margin', 'target_runs', 'target_overs', 'super_over',
       'method', 'umpire1', 'umpire2'],
      dtype='object')

In [54]:
df=pd.merge(deliveries, matches[['id','city','target_runs','target_overs','winner']], left_on ='match_id', right_on='id')

In [55]:
df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,id,city,target_runs,target_overs,winner
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,...,legbyes,0,NaN,NaN,NaN,335982,Bangalore,223.0,20.0,Kolkata Knight Riders
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,NaN,0,NaN,NaN,NaN,335982,Bangalore,223.0,20.0,Kolkata Knight Riders
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,...,wides,0,NaN,NaN,NaN,335982,Bangalore,223.0,20.0,Kolkata Knight Riders
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,...,NaN,0,NaN,NaN,NaN,335982,Bangalore,223.0,20.0,Kolkata Knight Riders
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,...,NaN,0,NaN,NaN,NaN,335982,Bangalore,223.0,20.0,Kolkata Knight Riders


In [56]:
df=df[df['inning']==2]#tracking only while chasing

In [57]:
df['curr_total'] = df.groupby('match_id').agg({'total_runs':'cumsum'})

In [58]:
df['runs_left'] = df['target_runs']-df['curr_total'] #calculating runs left

In [59]:
df['balls_left'] = 120-(df['over']*6+df['ball']) # balls left after each ball is thrown

In [60]:
df['total_wkts_down'] = df.groupby('match_id').agg({'is_wicket':'cumsum'})

In [61]:
df['wickets_left'] = 10- df['total_wkts_down']# wickets left after each ball

In [62]:
df['crr'] = (df['curr_total']*6)/(120-df['balls_left'])

In [68]:
#df[df['match_id']==335986][['total_runs','curr_total','balls_left','runs_left','over','crr']].tail()

,total_runs,curr_total,balls_left,runs_left,over,crr
1173,1,105,10,6.0,18,5.727273
1174,0,105,9,6.0,18,5.675676
1175,1,106,8,5.0,18,5.678571
1176,0,106,7,5.0,18,5.628319
1177,6,112,6,-1.0,18,5.894737


In [69]:
df['rrr'] = (df['runs_left']*6)/(df['balls_left'])# doubting here

In [73]:
df['result'] = df.apply(lambda x : 1 if x['winner']==x['batting_team'] else 0,axis=1)

In [76]:
final=df[['city','batting_team','bowling_team','runs_left','balls_left','wickets_left','target_runs','crr','rrr','result']]

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [78]:
x_train,x_test,y_train,y_test=train_test_split(final.drop(columns=['result']),final['result'], train_size=0.8, random_state=42)

In [ ]:
trf=ColumnTransformer([
    ('trf1',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
],remainder='passthrough')

In [ ]:
# def match_progression(x_df,match_id,pipe):
#     match = x_df[x_df['match_id'] == match_id]
#     match = match[(match['ball'] == 6)]
#     temp_df = match[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr']].dropna()
#     temp_df = temp_df[temp_df['balls_left'] != 0]
#     result = pipe.predict_proba(temp_df)
#     temp_df['lose'] = np.round(result.T[0]*100,1)
#     temp_df['win'] = np.round(result.T[1]*100,1)
#     temp_df['end_of_over'] = range(1,temp_df.shape[0]+1)
    
#     target = temp_df['total_runs_x'].values[0]
#     runs = list(temp_df['runs_left'].values)
#     new_runs = runs[:]
#     runs.insert(0,target)
#     temp_df['runs_after_over'] = np.array(runs)[:-1] - np.array(new_runs)
#     wickets = list(temp_df['wickets'].values)
#     new_wickets = wickets[:]
#     new_wickets.insert(0,10)
#     wickets.append(0)
#     w = np.array(wickets)
#     nw = np.array(new_wickets)
#     temp_df['wickets_in_over'] = (nw - w)[0:temp_df.shape[0]]
    
#     print("Target-",target)
#     temp_df = temp_df[['end_of_over','runs_after_over','wickets_in_over','lose','win']]
#     return temp_df,target
    